### Импортируем библиотеки

In [1]:
import pickle
import numpy as np
import pandas as pd
from docx import Document

### Считываем исходную таблицу из файла

In [2]:
def parse_docx_table(filename: str):
    document = Document(filename)

    tables = []
    for table in document.tables:
        # список списков с пустыми строками - структура будущего датафрейма
        df = [['' for x in range(len(table.columns))] for y in range(len(table.rows))]
        # заполняем дф данными из табл
        for x, row in enumerate(table.rows):
            for y, cell in enumerate(row.cells):
                if cell.text:
                    df[x][y] = cell.text
        tables.append(pd.DataFrame(df))

    return tables

In [3]:
# берем первую таблицу из списка
init_df = parse_docx_table('DE_task_table.docx')[0]
# заголовк таблицы - первый ряд
header = init_df.iloc[0]
# убираем первый ряд из таблицы
init_df = init_df[1:]
# назначаем название колонок дф
init_df.columns = header

In [4]:
# переводим формат дат, год 9999 заменяем на nan обработкой ошибок
init_df['start_date']= pd.to_datetime(init_df['start_date'], format="%d.%m.%Y", errors = 'coerce',)
init_df['finish_date']= pd.to_datetime(init_df['finish_date'], format="%d.%m.%Y", errors = 'coerce',)

In [5]:
# правило №2
init_df.loc[init_df['finish_date'].isnull(), 'finish_date'] = init_df['end_da']

/home/varsey-fabuza/PycharmProjects/dashboard-formation/env/lib/python3.10/site-packages/pandas/core/arrays/datetimes.py:339: UserWarning: Parsing '31.10.2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return cls._from_sequence_not_strict(scalars, dtype=dtype, copy=copy)
/home/varsey-fabuza/PycharmProjects/dashboard-formation/env/lib/python3.10/site-packages/pandas/core/arrays/datetimes.py:339: UserWarning: Parsing '30.09.2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return cls._from_sequence_not_strict(scalars, dtype=dtype, copy=copy)


In [6]:
init_df

,tab_num,start_date,finish_date,wday_type01,wday_type02,wday_type03,wday_type04,wday_type05,wplace_type,end_da
1,15123,2020-09-02,2020-10-31,0,0,0,0,0,0,31.10.2020
2,16234,2020-09-20,2020-10-30,0,0,1,1,0,2,
3,17345,2020-09-28,2020-10-25,1,0,0,0,0,2,
4,17345,2020-10-26,2020-12-31,1,1,1,1,1,1,
5,18456,2020-09-02,2020-09-30,2,2,2,2,2,3,30.09.2020
6,19567,2020-09-02,2020-12-31,3,3,3,3,3,4,


In [7]:
# словарь для каждого номера ряда исходной таблицы с датой старта и окончания режима
start_date_dict = dict(zip(range(init_df.shape[0]), init_df.start_date.to_list()))
finish_date_dict = dict(zip(range(init_df.shape[0]), init_df.finish_date.to_list()))

In [8]:
dash = pd.DataFrame()
# для каждого ряда - свой датафрейм по правилам №1 и №2
for num, tab_num in enumerate(init_df.tab_num.to_list()):
    interim = pd.DataFrame()
    interim['ymd_date'] = pd.date_range(start='2020-09-01', end='2020-12-31', freq='D')
    # дополнительные фичи для правила №8
    interim['weekday'] = interim['ymd_date'].dt.dayofweek
    interim['week'] = interim['ymd_date'].dt.isocalendar().week
    interim['week'] = interim['week'] - interim['week'].min()
    interim['halfweek'] = interim['week'] // 2
    interim['tab_num'] = tab_num
    interim['start_date'] = start_date_dict[num]
    interim['finish_date'] = finish_date_dict[num]
    dash = pd.concat([dash, interim])
dash['to_be_at_office'] = np.nan

In [9]:
repeated_tab_num = [x for x in init_df[init_df['tab_num'].duplicated()].tab_num.values]
repeated_tab_num

['17345']

In [10]:
def process_duplicated_tab_num(init_df: pd.DataFrame, repeated_tab_nums: list):
    dash_merged = init_df.copy()
    for tab_num in repeated_tab_nums:
        dash_merged = pd.concat(
            [
                dash_merged[~dash_merged.tab_num.isin([tab_num])],
                pd.merge(
                    dash_merged.loc[(dash_merged['tab_num'] == tab_num)][['ymd_date', 'weekday', 'tab_num']],
                    dash_merged.loc[
                        (dash_merged['tab_num'] == tab_num) &
                        (dash_merged['ymd_date'] <= dash_merged['finish_date']) &
                        (dash_merged['ymd_date'] >= dash_merged['start_date'])
                    ],
                    how='left'
                ).drop_duplicates()
            ]
        )
        print(dash_merged.shape)
    return dash_merged

In [11]:
dash_merged = process_duplicated_tab_num(dash, repeated_tab_num)

(610, 8)


In [12]:
dash.shape, dash_merged.shape, 122*5

((732, 8), (610, 8), 610)

In [13]:
assert dash_merged.shape[0] == 122 * len(dash.tab_num.unique())

In [14]:
dash_merged.sample(5)

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
48,2020-10-19,0,7,3,16234,2020-09-20,2020-10-30,NaN
35,2020-10-06,1,5,2,16234,2020-09-20,2020-10-30,NaN
87,2020-11-27,4,12,6,16234,2020-09-20,2020-10-30,NaN
8,2020-09-09,2,1,0,18456,2020-09-02,2020-09-30,NaN
33,2020-10-04,6,4,2,17345,2020-09-28,2020-10-25,NaN


In [15]:
with open('dash_merged.pickle', 'wb') as handle:
    pickle.dump(dash_merged, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('init_df.pickle', 'wb') as handle:
    pickle.dump(init_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
import pickle
import pandas as pd

In [3]:
with open("dash_merged.pickle", "rb") as handle:
    dash_merged = pickle.load(handle)

with open("init_df.pickle", "rb") as handle:
    init_df = pickle.load(handle)

In [4]:
init_df.shape, dash_merged.shape

((6, 10), (610, 8))

In [7]:
dash_merged

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,15123,2020-09-02,2020-10-31,NaN
1,2020-09-02,2,0,0,15123,2020-09-02,2020-10-31,NaN
2,2020-09-03,3,0,0,15123,2020-09-02,2020-10-31,NaN
3,2020-09-04,4,0,0,15123,2020-09-02,2020-10-31,NaN
4,2020-09-05,5,0,0,15123,2020-09-02,2020-10-31,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,17345,2020-10-26,2020-12-31,NaN
118,2020-12-28,0,17,8,17345,2020-10-26,2020-12-31,NaN
119,2020-12-29,1,17,8,17345,2020-10-26,2020-12-31,NaN
120,2020-12-30,2,17,8,17345,2020-10-26,2020-12-31,NaN


In [8]:
from rule_parser import RuleParser
rp = RuleParser(data=dash_merged, init_table=init_df)
wplace_type_dict = dict(zip(range(init_df.shape[0]), [int(x) for x in init_df.wplace_type.to_list()]))

In [13]:
data_processed = rp.transfrom(tab_num_index=4)


wtype 3


In [14]:
data_processed[data_processed['tab_num'] == init_df.tab_num.to_list()[4]]

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,18456,2020-09-02,2020-09-30,0.0
1,2020-09-02,2,0,0,18456,2020-09-02,2020-09-30,0.0
2,2020-09-03,3,0,0,18456,2020-09-02,2020-09-30,0.0
3,2020-09-04,4,0,0,18456,2020-09-02,2020-09-30,0.0
4,2020-09-05,5,0,0,18456,2020-09-02,2020-09-30,0.0
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,18456,2020-09-02,2020-09-30,0.0
118,2020-12-28,0,17,8,18456,2020-09-02,2020-09-30,0.0
119,2020-12-29,1,17,8,18456,2020-09-02,2020-09-30,0.0
120,2020-12-30,2,17,8,18456,2020-09-02,2020-09-30,0.0


In [8]:
rp.transfrom(tab_num_index=0)
# rp.transfrom(tab_num_index=3)
# rp.transfrom(tab_num_index=4)


wtype 0
wtype 1


,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,15123,2020-09-02,2020-10-31,0.0
1,2020-09-02,2,0,0,15123,2020-09-02,2020-10-31,1.0
2,2020-09-03,3,0,0,15123,2020-09-02,2020-10-31,1.0
3,2020-09-04,4,0,0,15123,2020-09-02,2020-10-31,1.0
4,2020-09-05,5,0,0,15123,2020-09-02,2020-10-31,0.0
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,17345,2020-10-26,2020-12-31,0.0
118,2020-12-28,0,17,8,17345,2020-10-26,2020-12-31,0.0
119,2020-12-29,1,17,8,17345,2020-10-26,2020-12-31,0.0
120,2020-12-30,2,17,8,17345,2020-10-26,2020-12-31,0.0


In [41]:
rp.data[rp.data['tab_num'] == init_df.tab_num.to_list()[4]]

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,18456,2020-09-02,2020-09-30,0.0
1,2020-09-02,2,0,0,18456,2020-09-02,2020-09-30,0.0
2,2020-09-03,3,0,0,18456,2020-09-02,2020-09-30,0.0
3,2020-09-04,4,0,0,18456,2020-09-02,2020-09-30,0.0
4,2020-09-05,5,0,0,18456,2020-09-02,2020-09-30,0.0
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,18456,2020-09-02,2020-09-30,0.0
118,2020-12-28,0,17,8,18456,2020-09-02,2020-09-30,0.0
119,2020-12-29,1,17,8,18456,2020-09-02,2020-09-30,0.0
120,2020-12-30,2,17,8,18456,2020-09-02,2020-09-30,0.0


In [23]:
start_date_dict = dict(zip(range(init_df.shape[0]), init_df.start_date.to_list()))
finish_date_dict = dict(zip(range(init_df.shape[0]), init_df.finish_date.to_list()))

In [24]:
init_df.tab_num.unique()[0]

'15123'

In [25]:
print(init_df.tab_num.to_list()[0])
tab_num_index = 0
dash_merged.loc[
    (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
    (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['weekday'] != 5) &
    (dash_merged['weekday'] != 6),
    'to_be_at_office'
] = 1

15123


In [26]:
dash_merged[dash_merged['tab_num'] == '15123']

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,15123,2020-09-02,2020-10-31,NaN
1,2020-09-02,2,0,0,15123,2020-09-02,2020-10-31,1.0
2,2020-09-03,3,0,0,15123,2020-09-02,2020-10-31,1.0
3,2020-09-04,4,0,0,15123,2020-09-02,2020-10-31,1.0
4,2020-09-05,5,0,0,15123,2020-09-02,2020-10-31,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,15123,2020-09-02,2020-10-31,NaN
118,2020-12-28,0,17,8,15123,2020-09-02,2020-10-31,NaN
119,2020-12-29,1,17,8,15123,2020-09-02,2020-10-31,NaN
120,2020-12-30,2,17,8,15123,2020-09-02,2020-10-31,NaN


In [27]:
print(init_df.tab_num.to_list()[3])

17345


In [28]:
tab_num_index = 3
dash_merged.loc[
    (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
    (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['weekday'] != 5) &
    (dash_merged['weekday'] != 6),
    'to_be_at_office'
] = 0

In [29]:
dash_merged[dash_merged['tab_num'] == '17345']

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,<NA>,<NA>,17345,NaT,NaT,NaN
1,2020-09-02,2,<NA>,<NA>,17345,NaT,NaT,NaN
2,2020-09-03,3,<NA>,<NA>,17345,NaT,NaT,NaN
3,2020-09-04,4,<NA>,<NA>,17345,NaT,NaT,NaN
4,2020-09-05,5,<NA>,<NA>,17345,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,17345,2020-10-26,2020-12-31,NaN
118,2020-12-28,0,17,8,17345,2020-10-26,2020-12-31,0.0
119,2020-12-29,1,17,8,17345,2020-10-26,2020-12-31,0.0
120,2020-12-30,2,17,8,17345,2020-10-26,2020-12-31,0.0


In [30]:
init_df[['wday_type01', 'wday_type02', 'wday_type03', 'wday_type04', 'wday_type05']][2:3].where('wday_type01 > 0') #.map({1 : 0, 1: 0})

ValueError: Array conditional must be same shape as self

In [ ]:
tab_num_index = 2
dash_merged.loc[
    (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
    (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['weekday'] != 5) &
    (dash_merged['weekday'] != 6) &
    (dash_merged['weekday'] == 0),
    'to_be_at_office'
] = 0

In [31]:
tab_num_index = 2
dash_merged.loc[
    (
        (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] < finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] > start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    ) &
    (
        (dash_merged['weekday'].isin([1, 2, 3, 4]))
    ),
    'to_be_at_office'
] = 1

In [32]:
tab_num_index = 1
dash_merged.loc[
    (
        (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    ) &
    (
        (dash_merged['weekday'].isin([0, 1, 4]))
    ),
    'to_be_at_office'
] = 1

In [33]:
tab_num_index = 1
dash_merged.loc[
    (
        (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([2, 3]))
    ),
    'to_be_at_office'
] = 0

In [34]:
dash_merged[dash_merged['tab_num'] == init_df.tab_num.to_list()[1]]

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,16234,2020-09-20,2020-10-30,NaN
1,2020-09-02,2,0,0,16234,2020-09-20,2020-10-30,NaN
2,2020-09-03,3,0,0,16234,2020-09-20,2020-10-30,NaN
3,2020-09-04,4,0,0,16234,2020-09-20,2020-10-30,NaN
4,2020-09-05,5,0,0,16234,2020-09-20,2020-10-30,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,16234,2020-09-20,2020-10-30,NaN
118,2020-12-28,0,17,8,16234,2020-09-20,2020-10-30,NaN
119,2020-12-29,1,17,8,16234,2020-09-20,2020-10-30,NaN
120,2020-12-30,2,17,8,16234,2020-09-20,2020-10-30,NaN


In [35]:
tab_num_index = 4
dash_merged.loc[
    (
        (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([0, 1, 2, 3, 4]))
    )
            &
    (
        (dash_merged['week'] % 2 == 0)
    ),
    'to_be_at_office'
] = 0

In [36]:
tab_num_index = 4
dash_merged.loc[
    (
        (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([0, 1, 2, 3, 4]))
    )
            &
    (
        (dash_merged['week'] % 2 == 1)
    ),
    'to_be_at_office'
] = 1

In [37]:
dash_merged[dash_merged['tab_num'] == init_df.tab_num.to_list()[4]]

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,18456,2020-09-02,2020-09-30,NaN
1,2020-09-02,2,0,0,18456,2020-09-02,2020-09-30,0.0
2,2020-09-03,3,0,0,18456,2020-09-02,2020-09-30,0.0
3,2020-09-04,4,0,0,18456,2020-09-02,2020-09-30,0.0
4,2020-09-05,5,0,0,18456,2020-09-02,2020-09-30,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,18456,2020-09-02,2020-09-30,NaN
118,2020-12-28,0,17,8,18456,2020-09-02,2020-09-30,NaN
119,2020-12-29,1,17,8,18456,2020-09-02,2020-09-30,NaN
120,2020-12-30,2,17,8,18456,2020-09-02,2020-09-30,NaN


In [38]:
dash_merged

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,15123,2020-09-02,2020-10-31,NaN
1,2020-09-02,2,0,0,15123,2020-09-02,2020-10-31,1.0
2,2020-09-03,3,0,0,15123,2020-09-02,2020-10-31,1.0
3,2020-09-04,4,0,0,15123,2020-09-02,2020-10-31,1.0
4,2020-09-05,5,0,0,15123,2020-09-02,2020-10-31,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,17345,2020-10-26,2020-12-31,NaN
118,2020-12-28,0,17,8,17345,2020-10-26,2020-12-31,0.0
119,2020-12-29,1,17,8,17345,2020-10-26,2020-12-31,0.0
120,2020-12-30,2,17,8,17345,2020-10-26,2020-12-31,0.0


In [ ]:
tab_num_index = 5
dash_merged.loc[
    (
        (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([0, 1, 2, 3, 4]))
    )
            &
    (
        (dash_merged['halfweek'] % 2 == 0)
    ),
    'to_be_at_office'
] = 0

In [ ]:
tab_num_index = 5
dash_merged.loc[
    (
        (dash_merged['tab_num'] == init_df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([0, 1, 2, 3, 4]))
    )
            &
    (
        (dash_merged['halfweek'] % 2 == 1)
    ),
    'to_be_at_office'
] = 1

In [ ]:
dash_merged[dash_merged['tab_num'] == init_df.tab_num.to_list()[5]]

In [ ]:
# TO-DO поменять формат даты

In [ ]:
dash_merged[['tab_num', 'ymd_date', 'to_be_at_office']].sort_values(by=['tab_num', 'ymd_date']).reset_index(drop=True)